# Solving a mincost k-SAT problem using QAOA

Before looking at this notebook, you should understand the methods described in the qaoa_mincost_sat notebook, as we will consider those as given here.
With the methods described in the qaoa_mincost_sat notebook, we are only able to solve 2-SAT problems, because we transform our SAT problem to a QUBO problem, which is limited to quadratic terms, and then implement the corresponding Ising model in a quantum circuit.
There are also methods for quadratization of Polynomial Unconstrained Binary Optimization (PUBO) problems, meaning a transformation to a QUBO problem, but they introduce auxiliary variables and thus require additional qubits. Nevertheless, we will also have a look at this method in this notebook.

The following describes, how to convert an arbitrary SAT problem to a PUBO problem and how to implement the corresponding problem circuit for applying QAOA, while using the same number of qubits as there are boolean variables in the SAT term.

The system, we will use as an example in this notebook, is described by the following boolean formular:
$
(x_1 \vee x_2) \wedge (x_2 \vee \neg x_3 \vee x_4) \wedge (x_3 \vee \neg x_5 \vee \neg x_6)
$

We will use the same implementation cost for each feature as in the other notebook:

| Feature | Cost $c_i$ |
|---------|------------|
| $x_1$   | 30         |
| $x_2$   | 20         |
| $x_3$   | 25         |
| $x_4$   | 50         |
| $x_5$   | 10         |
| $x_6$   | 10         |

## Converting k-SAT to PUBO

To get the penalties for general clauses $(x_i \vee \dots \vee x_j \vee \neg x_k \vee \dots \vee \neg x_l)$, we can use the following term:

$$(1 - x_i) \dots (1 - x_j) (x_k) \dots (x_l)$$

If the clause consists of exactly two boolean variables, this is equivalent to the quadratic penalties described by [Glover](https://arxiv.org/abs/1811.11538) we used in the other notebook.

Using this method we can transform our example into the corresponding PUBO model.

$$
y(\vec x) = (1 - x_1)(1 - x_2) + (1 - x_2)(x_3)(1 - x_4) + (1 - x_3)(x_5)(x_6)\\
= 1 - x_1 - x_2 + x_3 + x_1 x_2 - x_2 x_3 - x_3 x_4 + x_5 x_6 + x_2 x_3 x_4 - x_3 x_5 x_6
$$

We can now transform this PUBO model into an Ising model

$$
y(\vec z) = 1 - \frac{1 -z_1}{2} - \frac{1 - z_2}{2} + \frac{1 - z_3}{2} + \frac{1 - z_1}{2} \frac{1 - z_2}{2} - \frac{1 - z_2}{2} \frac{1 - z_3}{2} - \frac{1 - z_3}{2} \frac{1 - z_4}{2} + \frac{1 - z_5}{2} \frac{1 - z_6}{2} + \frac{1 - z_2}{2} \frac{1 - z_3}{2} \frac{1 - z_4}{2} - \frac{1 - z_3}{2} \frac{1 - z_5}{2} \frac{1 - z_6}{2}\\
= \frac{1}{2} - \frac{1}{4} z_1 + \frac{3}{8} z_2 + \frac{1}{8} z_4 - \frac{1}{8} z_5 - \frac{1}{8} z_6 + \frac{1}{4} z_1 z_2 - \frac{1}{8} z_2 z_3 + \frac{1}{8} z_2 z_4 - \frac{1}{8} z_3 z_4 - \frac{1}{8} z_3 z_5 - \frac{1}{8} z_3 z_6 + \frac{1}{8} z_5 z_6 - \frac{1}{8} z_2 z_3 z_4 + \frac{1}{8} z_3 z_5 z_6\\
$$

Which leaves us with the following validity hamiltonian $H_v$:

$$
H_v = \frac{1}{2} - \frac{1}{4} \sigma_1^z + \frac{3}{8} \sigma_2^z + \frac{1}{8} \sigma_4^z - \frac{1}{8} \sigma_5^z - \frac{1}{8} \sigma_6^z + \frac{1}{4} \sigma_1^z \sigma_2^z - \frac{1}{8} \sigma_2^z \sigma_3^z + \frac{1}{8} \sigma_2^z \sigma_4^z - \frac{1}{8} \sigma_3^z \sigma_4^z - \frac{1}{8} \sigma_3^z \sigma_5^z - \frac{1}{8} \sigma_3^z \sigma_6^z + \frac{1}{8} \sigma_5^z \sigma_6^z - \frac{1}{8} \sigma_2^z \sigma_3^z \sigma_4^z + \frac{1}{8} \sigma_3^z \sigma_5^z \sigma_6^z\\
$$

We can implement such a hamiltonian using the `qubovert` library.

## Quantum Circuit

We start by defining a parametrized circuit.
We will skip the details of initialization and implementing the mixing operator, as they are the same as in the other notebook.

In [ ]:
# Notebook Setup
from IPython.core import page
page.page = print

# Imports used for examples
from pprint import pprint

### Phase-separating operator
Like in the other notebook, the phase-separating operator $U_C$ encodes $C$ and can be derived from a cost Hamiltonian $H_{C}$ in Ising-form.
Unlike in the other notebook, there is now the possibility of having terms of degree higher than 2 in the cost function, for which we need to implement the corresponding multicontrolled $R_z$ gates.
We do this by decomposing the multicontrolled $R_z$ gate into multiple $CNOT$ gates and a single $R_z$ gate as illustrated by [Glos et al](https://arxiv.org/pdf/2009.07309.pdf).


In [ ]:
from qaoa_mincost_k_sat import k_rz_gate
from qiskit import QuantumCircuit

qubits = [0, 1, 2]
qc = QuantumCircuit(len(qubits))
qc_zzz = k_rz_gate(qc, qubits, 1)
qc_zzz.draw(output='mpl')

Using this implementation of multicontrolled $R_z$ gates, we can now implement the phase-separating operator for our example.

In [ ]:
# Hamiltonians may be described as a dict of tuples describing acting qubits and a value for each clause
# hamiltonian = {(q1?, q2?, ...) : factor}
sat_hamiltonian = {
    (): 0.5,
    (0,): 0.25,
    (1,): 0.375,
    (3,): 0.125,
    (4,): -0.125,
    (5,): -0.125,
    (0, 1): 0.25,
    (1, 2): -0.125,
    (1, 3): 0.125,
    (2, 3): -0.125,
    (2, 4): -0.125,
    (2, 5): -0.125,
    (4, 5): 0.125,
    (1, 2, 3): -0.125,
    (2, 4, 5): 0.125
}

In [ ]:
from qaoa_mincost_k_sat import problem_circuit
%psource problem_circuit

In [ ]:
from qiskit.circuit import Parameter

example_qc_problem = problem_circuit(sat_hamiltonian, 6, Parameter("$\\gamma$"))
example_qc_problem.draw(output='mpl')

In [ ]:
# cost and sat individually
from configproblem.util.model_transformation import convert_to_penalty
from qubovert import boolean_var

# define binary vars
x1, x2, x3, x4, x5, x6 = boolean_var('x1'), boolean_var('x2'), boolean_var('x3'), boolean_var('x4'), boolean_var('x5'), boolean_var('x6')

# SAT Penalty
alpha_sat = 1000 # 1e6

sat_instance = [[(x1, True), (x2, True)],
              [(x2, True), (x3, False), (x4, True)],
              [(x3, True), (x5, False), (x6, False)]]

# SAT PUBO
sat_model = convert_to_penalty(sat_instance)

# Cost PUBO
cost_model = 30*x1 + 20*x2 + 25*x3 + 50*x4 + 10*x5 + 10*x6

# Combine models
combined_model = cost_model + alpha_sat * sat_model
print("PUBO Combined Model:")
pprint(combined_model)
print("Ising Combined Model: ")
combined_hamiltonian = combined_model.to_puso()
pprint(combined_hamiltonian)

In [ ]:
from configproblem.util.visualization import plot_beta_gamma_cost_landscape, plot_f_mu_cost_landscape
import configproblem.qaoa.qaoa_mixer as mixer
# Plot cost landscape for different values of beta and gamma
hamiltonians = [{'hamiltonian': sat_model.to_puso(), 'name': 'SAT'},
               {'hamiltonian': cost_model.to_puso(), 'name': 'COST'},
               {'hamiltonian': combined_hamiltonian, 'name': 'COMBINED'}]
strategies = ['min', 'avg', 'top']
# plot_beta_gamma_cost_landscape(problem_circuit, mixer.standard_mixer, hamiltonians, strategies, 6, 0.2)
# 
# # Plot cost landscape for different f and mu
# plot_f_mu_cost_landscape(combined_hamiltonian, 6)

In [ ]:
from qaoa_application import apply_qaoa
from qaoa_mincost_k_sat import problem_circuit
from configproblem.util.hamiltonian_math import get_hamiltonian_dimension
import configproblem.qaoa.qaoa_parameter_optimization as parameter_optimization

hamiltonian = combined_hamiltonian
mixer_circuit = mixer.standard_mixer
parameter_optimization = parameter_optimization.get_optimizer('CG')
layers = 10 # more layers = higher approximation rate but more quantum errors when running on real qpu
n_features = 6
shots = 256
theta = {"beta": 0.01, "gamma": -0.01} # start values for optimization
strategy = 'avg'
use_warmstart = False
use_optimizer = True

if not use_warmstart:
    warmstart_statevector = None

counts, qc = apply_qaoa(problem_circuit, mixer_circuit, parameter_optimization, hamiltonian, layers, get_hamiltonian_dimension(hamiltonian), shots, theta, warmstart_statevector, strategy=strategy, use_optimizer=use_optimizer)

In [ ]:
from configproblem.util.visualization import plot_counts_histogram

best_config = "000010" # 654321
valid_configs = ["111111", "011111", "101111", "001111", "110111", "010111", "100111", "000111", "011011", "101011", "001011", "010011", "100011", "000011", "111101", "011101", "101101", "001101", "011001", "101001", "001001", "010001", "100001", "000001", "111110", "011110", "101110", "001110", "110110", "010110", "100110", "000110", "011010", "101010", "001010", "010010", "100010", "000010"]

plot_counts_histogram(counts, get_hamiltonian_dimension(hamiltonian), best_config, valid_configs)

In [ ]:
# Visualize results using the StatevectorSimulator
from qaoa_application import apply_qaoa_statevector
import numpy as np

probabilities, _ = apply_qaoa_statevector(problem_circuit, mixer_circuit, parameter_optimization, hamiltonian, layers, get_hamiltonian_dimension(hamiltonian), theta, warmstart_statevector, strategy=strategy, use_optimizer=use_optimizer, print_res=True)
probabilities_dict = {}
for i in range(0, 2 ** get_hamiltonian_dimension(hamiltonian)):
    probabilities_dict[(np.binary_repr(i, width=get_hamiltonian_dimension(hamiltonian)))] = round(probabilities[i], 4)

plot_counts_histogram(probabilities_dict, get_hamiltonian_dimension(hamiltonian), best_config, valid_configs)

## Quadratization of PUBOs

Instead of using the PUBO model directly, we can also quadratize the model and use the corresponding QUBO model.
For example, if we transform the clause $(x_i \vee x_j \vee x_k)$, we get the following penalty:

$$1 - x_i - x_j - x_k + x_i x_j + x_i x_k + x_j x_k - x_i x_j x_k$$

By introducing a new auxiliary variable $w_{ij} = x_i x_j$ we can rewrite this penalty as:

$$1 - x_i - x_j - x_k + w_{ij} + x_i x_k + x_j x_k - w_{ij} x_k$$

This term no longer contains a product of three variables but products of at most two variables and can therefore be used in a QUBO model.
The method for reduction by substitution above was first described by I. G. Rosenberg in "Reduction of Bivalent Maximization to the Quadratic Case" (1975) and can be found in a collection of different quadratization methods by [Dattani](https://arxiv.org/pdf/1901.04405.pdf).

We use the `to_quso()` function from the `qubovert` library to quadratize the PUBO model.
After applying QAOA to the resulting QUBO model, we can use the `convert_ancilla_bit_results()` function to convert the results back to the original PUBO model.

In [ ]:
# Visualize results using the StatevectorSimulator with quadratized hamiltonian
from qaoa_application import apply_qaoa_statevector
from qaoa_mincost_k_sat import convert_ancilla_bit_results
from qaoa_mincost_sat import problem_circuit as qubo_problem_circuit
import numpy as np

hamiltonian = combined_model.to_quso()

print("Quadratized Hamiltonian:")
pprint(hamiltonian)

probabilities, _ = apply_qaoa_statevector(qubo_problem_circuit, mixer_circuit, parameter_optimization, hamiltonian, layers, get_hamiltonian_dimension(hamiltonian), theta, warmstart_statevector, strategy=strategy, use_optimizer=use_optimizer, print_res=True)
probabilities_dict = {}
for i in range(0, 2 ** get_hamiltonian_dimension(hamiltonian)):
    probabilities_dict[np.binary_repr(i, width=get_hamiltonian_dimension(hamiltonian))] = probabilities[i]

probabilities_dict = convert_ancilla_bit_results(probabilities_dict, n_features)

for key in probabilities_dict:
    probabilities_dict[key] = round(probabilities_dict[key], 4)

plot_counts_histogram(probabilities_dict, n_features, best_config, valid_configs)

## Configuration Prioritization using PUBO models

If we have an arbitrary SAT instance, we can use QAOA to find the most likely configuration. We can then exclude the configuration we found from the SAT instance by adding the following clause:

$$(x_i \vee \dots \vee x_j \vee \neg x_k \vee \dots \vee \neg x_l)$$

Where $x_i$ to $x_j$ are the variables that have the value $0$ for the found configuration and $x_k$ to $x_l$ are the variables that have the value $1$ for the found configuration.

We can then run QAOA again with this new SAT instance to find the next most likely configuration.
We can repeat this process until we have found a certain number of configurations, thus creating a prioritized list of configurations.

In [ ]:
from configproblem.qaoa.qaoa_mincost_k_sat import exclude_config

list_length = 5
debug_output = True

current_sat_instance = sat_instance
boolean_vars = [x1, x2, x3, x4, x5, x6]
prioritized_list = []
current_hamiltonian = combined_model.to_puso()
for i in range(list_length):
    probabilities, _ = apply_qaoa_statevector(problem_circuit, mixer_circuit, parameter_optimization, current_hamiltonian, strategy=strategy, print_res=False)
    probabilities_dict = {}
    for j in range(0, 2 ** get_hamiltonian_dimension(current_hamiltonian)):
        probabilities_dict[(np.binary_repr(j, width=get_hamiltonian_dimension(current_hamiltonian)))] = round(probabilities[j], 4)

    current_config = max(probabilities_dict, key=probabilities_dict.get)
    current_sat_instance = exclude_config(current_sat_instance, boolean_vars, current_config)
    new_combined_model = cost_model + alpha_sat * convert_to_penalty(current_sat_instance)

    if debug_output:
        print("Current hamiltonian: " + str(current_hamiltonian))
        plot_counts_histogram(probabilities_dict, get_hamiltonian_dimension(current_hamiltonian), best_config, valid_configs)
        print("New combined model: " + str(new_combined_model))

    valid_configs.remove(current_config)
    current_hamiltonian = new_combined_model.to_puso()
    prioritized_list.append(current_config)

print(prioritized_list)